# ESG

## 民國110年、111年

In [3]:
import requests
from bs4 import BeautifulSoup as bs

stock_id = int(input("請輸入股票代碼: "))
year = int(input("請輸入民國年份: "))

data = {
    "encodeURIComponent": 1,
    "step": 2,
    "co_id": stock_id, # 可以是數字或文字
    "YEAR": year # 可以是數字或文字
}
response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)
response

<Response [200]>

In [4]:
import pandas as pd

pd.options.display.float_format = '{:.4f}'.format

target_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                 "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                 "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                 "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]
fields = []
values = []
soup = bs(response.text, features = "html.parser")
# print(soup.prettify())
tables = soup.find_all("table")
target_table = tables[-2]
trs = target_table.find_all("tr")
for tr in trs:
    td = tr.find("td", attrs = {"align": "center"})
    if td is None:
        continue
    text = td.text
    end = text.find("(")
    if end == -1:
        text = text.strip()
    else:    
        text = text[: end].strip()
    # print(text)
    # print("-" * 10)
    if text in target_fields:
        field = tr.find("td", attrs = {"align": "center"}).text.strip()
        # print(tr)
        # print(field)
        if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
            fields.extend(["女性董事席次(席)", "女性董事比率"])
            temp_list = tr.find_all("td", attrs = {"align": "right"})
            # print(temp_list)
            number = round(float(temp_list[0].text.strip()[: -1]), 4)
            ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
            # print(number, ratio)
            values.extend([number, ratio])
            # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
        else: 
            fields.append(field)
            value = tr.find("td", attrs = {"align": "right"})
            # print(value)
            if value is None:
                values.append("Undefined")
            elif value.text == "":
                values.append("Undefined")  
            else:
                value = value.text.strip().replace(",", "")
                if value[-1] == "%":
                    value = value[: -1]
                    value = round(float(value) / 100, 4)
                    # print(value)
                elif value[-1] == "人":
                    value = round(float(value[: -1]), 4)
                    # print(value)
                else:
                    value = round(float(value), 4)
                # print(value)
                values.append(value)

df_esg = pd.DataFrame([values], columns = fields).transpose()
df_esg.rename(columns = {0: "數值"})

,數值
直接溫室氣體排放量(範疇一) (噸CO2e),2018789.0000
能源間接(範疇二) (噸CO2e),9539765.0000
其他間接(範疇三) (噸CO2e),7502136.0000
溫室氣體排放密集度 (噸CO2e/百萬元營業額),5.1230
再生能源使用率,0.1040
用水量 (公噸),104681272.0000
用水密集度 (公噸/公升/十二吋晶圓當量－光罩數),137.3000
有害廢棄物 (公噸),401215.0000
非有害廢棄物 (公噸),342804.0000
總重量 (有害+非有害) (公噸),744019.0000


## 民國112年之後

In [5]:
import json
import requests

stock_id = input("請輸入股票代碼: ")

payload = {
  "companyCode": stock_id, # 要是字串
  "yearList": [2023],
  "year": 2023
}

response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)
                        
response

<Response [200]>

In [6]:
import pandas as pd

newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                 "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                 "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                 "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                 "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                 "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                 "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

data = response.json()["data"]
treeModels = data[0]["treeModels"]

names = []
values = []
temp_list = []
for i in range(len(treeModels)):
    items = treeModels[i]["items"]
    for j in range(len(items)):
        sections = items[j]["sections"]
        # print(sections)
        for k in range(len(sections)):
            name = sections[k]["name"]
            # print(name)
            if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                # print(name)
                if name == "女性董事比率":
                    names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                    value = sections[k]["controls"][0]["value"]
                    value = value[: -1]
                    value = round(float(value) / 100, 4)
                    temp_list.insert(0, value)
                    values.extend(temp_list)
                elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                    value = sections[k]["controls"][0]["value"]
                    # print(value)
                    value = value[: -1]
                    value = round(float(value) / 100, 4)
                    temp_list.append(value)
                else:
                    value = sections[k]["controls"][0]["value"]
                    value = value.replace(",", "")
                    if value[-1] == "%":
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                    value = round(float(value), 4)
                    values.append(value)
                    names.append(name)

df_new_esg = pd.DataFrame(data = [values], columns = names).transpose()
df_new_esg.rename(columns = {0: "數值"})

,數值
直接(範疇一)溫室氣體排放量(公噸CO₂e),1596031.2036
能源間接(範疇二)溫室氣體排放量(公噸CO₂e),10187386.7231
其他間接(範疇三)溫室氣體排放量(公噸CO₂e),7616654.7896
溫室氣體排放密集度(公噸CO₂e/百萬元營業額),5.2239
再生能源使用率,0.1116
用水量(公噸(t)),101001617.0000
用水密集度,176.4318
有害廢棄物(公噸(t)),371236.0000
非有害廢棄物(公噸(t)),285605.0000
總重量(有害+非有害)(公噸(t)),656841.0000


# ESG 資料合併

## 三年 ESG 資料合併

In [ ]:
json_data = {'company': {'name': '台積電',
 'stock_id': '2330',
 'environmental': [{'year': '2023', 'index': '直接溫室氣體排放量(範疇一) (噸CO2e)', 'value': 2151937.0000},
 {'index': '能源間接(範疇二) (噸CO2e)', 'value': 8152497.0000},
 {'index': '其他間接(範疇三) (噸CO2e)', 'value': 6049256.0000},
 {'index': '溫室氣體排放密集度 (噸CO2e/百萬 營業額)', 'value': 6.4910},
 {'index': '再生能源使用率', 'value': 0.0920},
 {'index': '用水量 (公噸)', 'value': 82823740.0000},
 {'index': '用水密集度 (公噸/單位產品用水量（公升／十二吋晶圓當量－光罩數）)', 'value': 119.9000},
 {'index': '有害廢棄物 (公噸)', 'value': 339623.0000},
 {'index': '非有害廢棄物 (公噸)', 'value': 335080.0000},
 {'index': '總重量 (有害+非有害) (公噸)', 'value': 674703.0000},
 {'index': '廢棄物密集度 (公噸/單位晶圓委外廢棄物處理量（公斤／十二吋晶圓當量－光罩數）)', 'value': 0.9900}],
 'social': [{'index': '員工福利平均數 (仟元/人)(每年6/2起公開)', 'value': 2706.0000},
 {'index': '員工薪資平均數 (仟元/人)(每年6/2起公開)', 'value': 2463.0000},
 {'index': '非擔任主管職務之全時員工薪資平均數 (仟元/人)(每年7/1起公開)', 'value': 2425.0000},
 {'index': '非擔任主管職務之全時員工薪資中位數 (仟元/人)(每年7/1起公開)', 'value': 1851.0000},
 {'index': '管理職女性主管占比', 'value': 0.1300},
 {'index': '職業災害人數', 'value': 44.0000},
 {'index': '職業災害人數比率 (職災人數/總人數)', 'value': 0.0007}],
 'governance': [{'index': '董事會席次 (席)', 'value': 10.0000},
 {'index': '獨立董事席次 (席)', 'value': 6.0000},
 {'index': '女性董事席次(席)', 'value': 1.0000},
 {'index': '女性董事比率', 'value': 0.1000},
 {'index': '董事出席董事會出席率', 'value': 1.0000},
 {'index': '董監事進修時數符合進修要點比率', 'value': 0.4000},
 {'index': '公司年度召開法說會次數(次)', 'value': 4.0000}]}
}
with open("2330_esg_3years.json", "w", encoding = "utf-8") as file:
    json.dump(dict_format, fp = file, ensure_ascii = False, indent = 2)

In [ ]:
# print(values)
# print(names)
dict_format = {"company": {"name": company_name,
                           "stock_id": stock_id,
                           "environmental": [{} for _ in range(33)],
                           "social": [{} for _ in range(21)],
                           "governance": [{} for _ in range(21)]}

} # 11 7 7 11 7 7 11 7 7
for i in range(len(values)): # 0 ~ 74
    epoch = i // 25 + 1
    # if epoch == 3:
    #     break
    j = i % 25 # 0 ~ 24
    value = values[i]
    name = names[j]
    if j // 11 < 1:
        if epoch == 1:
            dict_format["company"]["environmental"][j + (epoch - 1) * 11] = {"year": year, "index": name, "value": value}
        elif epoch == 2:
            dict_format["company"]["environmental"][j + (epoch - 1) * 11] = {"year": "2022", "index": name, "value": value}
        else:
            dict_format["company"]["environmental"][j + (epoch - 1) * 11] = {"year": "2021", "index": name, "value": value}
    elif j // 18 < 1:
        if epoch == 1:
            dict_format["company"]["social"][j % 11 + (epoch - 1) * 7] = {"year": year, "index": name, "value": value}
        elif epoch == 2:
            dict_format["company"]["social"][j % 11 + (epoch - 1) * 7] = {"year": "2022", "index": name, "value": value}
        else:
            dict_format["company"]["social"][j % 11 + (epoch - 1) * 7] = {"year": "2021", "index": name, "value": value}
    elif j // 25 < 1:
        if epoch == 1:
            dict_format["company"]["governance"][j % 18 + (epoch - 1) * 7] = {"year": year, "index": name, "value": value}
        elif epoch == 2:
            dict_format["company"]["governance"][j % 18 + (epoch - 1) * 7] = {"year": "2022", "index": name, "value": value}
        else:
            dict_format["company"]["governance"][j % 18 + (epoch - 1) * 7] = {"year": "2021", "index": name, "value": value}
            

dict_format

In [ ]:
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

def three_years_data():
    stock_id = input("請輸入股票代碼: ")
    year = "2023"
    payload = {
    "companyCode": stock_id, # 要是字串
    "yearList": [year],
    "year": year
    }

    newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

    newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                    "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                    "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                    "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                    "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                    "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                    "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

    oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                    "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                    "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                    "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]

    data = newest_response.json()["data"]
    company_name = data[0]["companyName"]
    treeModels = data[0]["treeModels"]

    names = []
    values = []
    temp_list = []
    for i in range(len(treeModels)):
        items = treeModels[i]["items"]
        for j in range(len(items)):
            sections = items[j]["sections"]
            # print(sections)
            for k in range(len(sections)):
                name = sections[k]["name"]
                # print(name)
                if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                    # print(name)
                    if name == "女性董事比率":
                        names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                        value = sections[k]["controls"][0]["value"]
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.insert(0, value)
                        values.extend(temp_list)
                    elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                        value = sections[k]["controls"][0]["value"]
                        # print(value)
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.append(value)
                    else:
                        value = sections[k]["controls"][0]["value"]
                        value = value.replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                        value = round(float(value), 4)
                        values.append(value)
                        names.append(name)


    def call_oldest_response(stock_id, year):
        data = {
            "encodeURIComponent": 1,
            "step": 2,
            "co_id": stock_id, # 可以是數字或文字
            "YEAR": year # 可以是數字或文字
        }
        oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

        fields = []
        # values = []
        soup = bs(oldest_response.text, features = "html.parser")
        # print(soup.prettify())
        tables = soup.find_all("table")
        target_table = tables[-2]
        trs = target_table.find_all("tr")
        for tr in trs:
            td = tr.find("td", attrs = {"align": "center"})
            if td is None:
                continue
            text = td.text
            end = text.find("(")
            if end == -1:
                text = text.strip()
            else:    
                text = text[: end].strip()
            # print(text)
            # print("-" * 10)
            if text in oldest_fields:
                field = tr.find("td", attrs = {"align": "center"}).text.strip()
                # print(tr)
                # print(field)
                if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                    fields.extend(["女性董事席次(席)", "女性董事比率"])
                    temp_list = tr.find_all("td", attrs = {"align": "right"})
                    # print(temp_list)
                    number = round(float(temp_list[0].text.strip()[: -1]), 4)
                    ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                    # print(number, ratio)
                    values.extend([number, ratio])
                    # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                else: 
                    fields.append(field)
                    value = tr.find("td", attrs = {"align": "right"})
                    # print(value)
                    if value is None:
                        values.append("Undefined")
                    elif value.text == "":
                        values.append("Undefined")  
                    else:
                        value = value.text.strip().replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                            # print(value)
                        elif value[-1] == "人":
                            value = round(float(value[: -1]), 4)
                            # print(value)
                        else:
                            value = round(float(value), 4)
                        # print(value)
                        values.append(value)

    call_oldest_response(stock_id, int(year) - 1911 - 1)
    time.sleep(1.5)
    call_oldest_response(stock_id, int(year) - 1911 - 2)
    # print(names)
    values[: 25], values[50: ] = values[50: ], values[: 25]
    dict_format = {
    "company": {
        "name": company_name,
        "stock_id": stock_id,
        "environmental": {
            "years": ["2021", "2022", "2023"],
            "categories": {
                "溫室氣體排放量": [
                    "直接(範疇一)溫室氣體排放量(公噸CO₂e)",
                    "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)",
                    "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                    "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)"
                ], 
                "再生能源使用率": [
                    "再生能源使用率"
                ], 
                "用水量": [
                    "用水量(公噸(t))",
                    "用水密集度"
                ],   
                "廢棄物重量": [
                    "有害廢棄物(公噸(t))",
                    "非有害廢棄物(公噸(t))",
                    "總重量(有害+非有害)(公噸(t))",
                    "廢棄物密集度"
                ]             
            }, 
            "data": {
                "直接(範疇一)溫室氣體排放量(公噸CO₂e)": [],
                "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)": [],
                "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)": [],
                "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)": [],
                "再生能源使用率": [],
                "用水量(公噸(t))": [],
                "用水密集度": [],
                "有害廢棄物(公噸(t))": [],
                "非有害廢棄物(公噸(t))": [],
                "總重量(有害+非有害)(公噸(t))": [],
                "廢棄物密集度": []
            }
        }, 
        "social": {
            "years": ["2021", "2022", "2023"], 
            "categories": {
                  "員工福利平均數": [
                      "員工福利平均數(每年6/2起公開)(仟元/人)"
                  ], 
                  "員工薪資平均數": [
                      "員工薪資平均數(每年6/2起公開)(仟元/人)"
                  ],
                  "非擔任主管職務之全時員工薪資": [
                      "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)",
                      "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)"
                  ],
                  "管理職女性主管占比": [
                      "管理職女性主管占比"
                  ],
                  "職業災害": [
                      "職業災害人數(人)",
                      "職業災害人數比率"
                  ]
            },
            "data": {
              "員工福利平均數(每年6/2起公開)(仟元/人)": [],
              "員工薪資平均數(每年6/2起公開)(仟元/人)": [],
              "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)": [],
              "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)": [],
              "管理職女性主管占比": [],
              "職業災害人數(人)": [],
              "職業災害人數比率": []
            }
        },
        "governance": {
            "years": ["2021", "2022", "2023"],
            "categories": {
                "董事會結構與會議": [
                    "董事會席次(席)",
                    "獨立董事席次(席)",
                    "女性董事席次(席)",
                    "公司年度召開法說會次數(次)"
                ],
                "董事參與與能力": [
                    "女性董事比率",
                    "董事出席董事會出席率",
                    "董事進修時數符合進修要點比率"
                ]
            },
            "data": {
                "董事會席次(席)": [],
                "獨立董事席次(席)": [],
                "女性董事席次(席)": [],
                "公司年度召開法說會次數(次)": [],
                "女性董事比率": [],
                "董事出席董事會出席率": [],
                "董事進修時數符合進修要點比率": []
                }
            }
        }
    }

    for i in range(len(values)): # 0 ~ 74
        value = values[i]
        j = i % 25 # 0 ~ 24
        name = names[j]
        # print(name)
        if j <= 10:
            dict_format["company"]["environmental"]["data"][name].append(value)
        elif j <= 17:
            dict_format["company"]["social"]["data"][name].append(value)
        else:
            dict_format["company"]["governance"]["data"][name].append(value)

    with open("2330_esg_3years_test.json", "w", encoding = "utf-8") as file:
        json.dump(dict_format, fp = file, ensure_ascii = False, indent = 2)

    return dict_format


    # return values

    # dict_format = {"company": {"name": company_name,
    #                         "stock_id": stock_id,
    #                         "environmental": [{} for _ in range(33)],
    #                         "social": [{} for _ in range(21)],
    #                         "governance": [{} for _ in range(21)]}

    # } # 11 7 7 11 7 7 11 7 7

    # for i in range(len(values)): # 0 ~ 74
    #     epoch = i // 25 + 1
    #     if epoch == 3:
    #         break
    #     j = i % 25 # 0 ~ 24
    #     value = values[i]
    #     name = names[j]
    #     if j // 11 < 1:
    #         if epoch == 1:
    #             dict_format["company"]["environmental"][j + (epoch - 1) * 11] = {"year": year, "index": name, "value": value}
    #         elif epoch == 2:
    #             dict_format["company"]["environmental"][j + (epoch - 1) * 11] = {"year": "2022", "index": name, "value": value}
    #         else:
    #             dict_format["company"]["environmental"][j + (epoch - 1) * 11] = {"year": "2021", "index": name, "value": value}
    #     elif j // 18 < 1:
    #         if epoch == 1:
    #             dict_format["company"]["social"][j % 11 + (epoch - 1) * 7] = {"year": year, "index": name, "value": value}
    #         elif epoch == 2:
    #             dict_format["company"]["social"][j % 11 + (epoch - 1) * 7] = {"year": "2022", "index": name, "value": value}
    #         else:
    #             dict_format["company"]["social"][j % 11 + (epoch - 1) * 7] = {"year": "2021", "index": name, "value": value}
    #     elif j // 25 < 1:
    #         if epoch == 1:
    #             dict_format["company"]["governance"][j % 18 + (epoch - 1) * 7] = {"year": year, "index": name, "value": value}
    #         elif epoch == 2:
    #             dict_format["company"]["governance"][j % 18 + (epoch - 1) * 7] = {"year": "2022", "index": name, "value": value}
    #         else:
    #             dict_format["company"]["governance"][j % 18 + (epoch - 1) * 7] = {"year": "2021", "index": name, "value": value}
    # return dict_format

dict_format = three_years_data()
dict_format

In [ ]:
{
  "company": {
    "name": "台積電",
    "stock_id": "2330",
    "environmental": {
      "years": ["2021", "2022", "2023"],
      "categories": {
        "溫室氣體排放量": [  
          "直接(範疇一)溫室氣體",
          "能源間接(範疇二)溫室氣體",
          "其他間接(範疇三)溫室氣體",
          "溫室氣體排放密集度"
        ],
        "再生能源使用率": [
          "再生能源使用率"
        ],
        "用水量": [
          "用水量(公噸(t))",
          "用水密集度"
        ],
        "廢棄物重量": [
          "有害廢棄物(公噸(t))",
          "非有害廢棄物(公噸(t))",
          "總重量(有害+非有害)(公噸(t))",
          "廢棄物密集度"
        ]
      },
      "data": {
        "直接(範疇一)溫室氣體排放量(公噸CO₂e)": [2151937, 2018789, 1596031.2036],
        "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)": [8152497, 9539765, 10187386.7231],
        "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)": [6049256, 7502136, 7616654.7896],
        "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)": [6.491, 5.123, 5.2239],
        "再生能源使用率": [0.092, 0.104, 0.1116],
        "用水量(公噸(t))": [82823740, 104681272, 101001617],
        "用水密集度": [119.9, 137.3, 176.43],
        "有害廢棄物(公噸(t))": [339623, 401215, 371236],
        "非有害廢棄物(公噸(t))": [335080, 342804, 2856],
        "總重量(有害+非有害)(公噸(t))": [674703, 744019, 6568],
        "廢棄物密集度": [0.99, 0.99, 1.17]
      }
    },
    "social": {
      "years": ["2021", "2022", "2023"],
      "categories": {
        "員工福利平均數": [
          "員工福利平均數(每年6/2起公開)(仟元/人)"
        ],
        "員工薪資平均數": [
          "員工薪資平均數(每年6/2起公開)(仟元/人)"
        ],
        "非擔任主管職務之全時員工薪資": [
          "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)",
          "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)"
        ],
        "管理職女性主管占比": [
          "管理職女性主管占比"
        ],
        "職業災害": [
          "職業災害人數(人)",
          "職業災害人數比率"
        ]
      },
      "data": {
        "員工福利平均數(每年6/2起公開)(仟元/人)": [2706, 3446, 3015],
        "員工薪資平均數(每年6/2起公開)(仟元/人)": [2463, 3175, 2737],
        "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)": [2425, 3167, 2842],
        "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)": [1851, 2435, 2248],
        "管理職女性主管占比": [0.13, 0.133, 0.141],
        "職業災害人數(人)": [44, 35, 48],
        "職業災害人數比率": [0.0007, 0.0005, 0.0007]
      }
    },
    "governance": {
      "years": ["2021", "2022", "2023"],
      "categories": {
        "董事會結構與會議": [
          "董事會席次(席)",
          "獨立董事席次(席)",
          "女性董事席次(席)",
          "公司年度召開法說會次數(次)"
        ],
        "董事參與與能力": [
          "女性董事比率",
          "董事出席董事會出席率",
          "董事進修時數符合進修要點比率"
        ]
      },
      "data": {
        "董事會席次(席)": [10, 10, 10],
        "獨立董事席次(席)": [6, 6, 6],
        "女性董事席次(席)": [1, 1, 1],
        "公司年度召開法說會次數(次)": [4, 5, 4],
        "女性董事比率": [0.1, 0.1, 0.1],
        "董事出席董事會出席率": [1, 1, 0.94],
        "董事進修時數符合進修要點比率": [0.4, 0.3, 0.2]
      }
    }
  }
}

In [9]:
a = [1, 2, 3, 4, 5, 6]
a[: 2], a[4: ] = a[4: ], a[: 2]
a

[5, 6, 3, 4, 1, 2]

In [ ]:
with open("2330_esg_3years_test.json", "w", encoding = "utf-8") as file:
    json.dump(dict_format, fp = file, ensure_ascii = False, indent = 2)

In [7]:
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from flask import Flask

app = Flask(__name__)

@app.route("/get_three_years_ESG_data_format", methods = ["get"])
def three_years_ESG_data_format():
    stock_id = input("請輸入股票代碼: ")
    year = "2023"
    payload = {
    "companyCode": stock_id, # 要是字串
    "yearList": [year],
    "year": year
    }

    newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

    newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                    "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                    "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                    "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                    "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                    "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                    "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

    oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                    "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                    "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                    "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]

    data = newest_response.json()["data"]
    company_name = data[0]["companyName"]
    treeModels = data[0]["treeModels"]

    names = []
    values = []
    temp_list = []
    for i in range(len(treeModels)):
        items = treeModels[i]["items"]
        for j in range(len(items)):
            sections = items[j]["sections"]
            # print(sections)
            for k in range(len(sections)):
                name = sections[k]["name"]
                # print(name)
                if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                    # print(name)
                    if name == "女性董事比率":
                        names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                        value = sections[k]["controls"][0]["value"]
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.insert(0, value)
                        values.extend(temp_list)
                    elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                        value = sections[k]["controls"][0]["value"]
                        # print(value)
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.append(value)
                    else:
                        value = sections[k]["controls"][0]["value"]
                        value = value.replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                        value = round(float(value), 4)
                        values.append(value)
                        names.append(name)


    def call_oldest_response(stock_id, year):
        data = {
            "encodeURIComponent": 1,
            "step": 2,
            "co_id": stock_id, # 可以是數字或文字
            "YEAR": year # 可以是數字或文字
        }
        oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

        fields = []
        # values = []
        soup = bs(oldest_response.text, features = "html.parser")
        # print(soup.prettify())
        tables = soup.find_all("table")
        target_table = tables[-2]
        trs = target_table.find_all("tr")
        for tr in trs:
            td = tr.find("td", attrs = {"align": "center"})
            if td is None:
                continue
            text = td.text
            end = text.find("(")
            if end == -1:
                text = text.strip()
            else:    
                text = text[: end].strip()
            # print(text)
            # print("-" * 10)
            if text in oldest_fields:
                field = tr.find("td", attrs = {"align": "center"}).text.strip()
                # print(tr)
                # print(field)
                if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                    fields.extend(["女性董事席次(席)", "女性董事比率"])
                    temp_list = tr.find_all("td", attrs = {"align": "right"})
                    # print(temp_list)
                    number = round(float(temp_list[0].text.strip()[: -1]), 4)
                    ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                    # print(number, ratio)
                    values.extend([number, ratio])
                    # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                else: 
                    fields.append(field)
                    value = tr.find("td", attrs = {"align": "right"})
                    # print(value)
                    if value is None:
                        values.append("Undefined")
                    elif value.text == "":
                        values.append("Undefined")  
                    else:
                        value = value.text.strip().replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                            # print(value)
                        elif value[-1] == "人":
                            value = round(float(value[: -1]), 4)
                            # print(value)
                        else:
                            value = round(float(value), 4)
                        # print(value)
                        values.append(value)

    call_oldest_response(stock_id, int(year) - 1911 - 1)
    time.sleep(1.5)
    call_oldest_response(stock_id, int(year) - 1911 - 2)
    # print(names)
    values[: 25], values[50: ] = values[50: ], values[: 25]
    dict_format = {
    "company": {
        "name": company_name,
        "stock_id": stock_id,
        "environmental": {
            "years": ["2021", "2022", "2023"],
            "categories": {
                "溫室氣體排放量": [
                    "直接(範疇一)溫室氣體排放量(公噸CO₂e)",
                    "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)",
                    "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                    "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)"
                ], 
                "再生能源使用率": [
                    "再生能源使用率"
                ], 
                "用水量": [
                    "用水量(公噸(t))",
                    "用水密集度"
                ],   
                "廢棄物重量": [
                    "有害廢棄物(公噸(t))",
                    "非有害廢棄物(公噸(t))",
                    "總重量(有害+非有害)(公噸(t))",
                    "廢棄物密集度"
                ]             
            }, 
            "data": {
                "直接(範疇一)溫室氣體排放量(公噸CO₂e)": [],
                "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)": [],
                "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)": [],
                "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)": [],
                "再生能源使用率": [],
                "用水量(公噸(t))": [],
                "用水密集度": [],
                "有害廢棄物(公噸(t))": [],
                "非有害廢棄物(公噸(t))": [],
                "總重量(有害+非有害)(公噸(t))": [],
                "廢棄物密集度": []
            }
        }, 
        "social": {
            "years": ["2021", "2022", "2023"], 
            "categories": {
                  "員工福利平均數": [
                      "員工福利平均數(每年6/2起公開)(仟元/人)"
                  ], 
                  "員工薪資平均數": [
                      "員工薪資平均數(每年6/2起公開)(仟元/人)"
                  ],
                  "非擔任主管職務之全時員工薪資": [
                      "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)",
                      "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)"
                  ],
                  "管理職女性主管占比": [
                      "管理職女性主管占比"
                  ],
                  "職業災害": [
                      "職業災害人數(人)",
                      "職業災害人數比率"
                  ]
            },
            "data": {
              "員工福利平均數(每年6/2起公開)(仟元/人)": [],
              "員工薪資平均數(每年6/2起公開)(仟元/人)": [],
              "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)": [],
              "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)": [],
              "管理職女性主管占比": [],
              "職業災害人數(人)": [],
              "職業災害人數比率": []
            }
        },
        "governance": {
            "years": ["2021", "2022", "2023"],
            "categories": {
                "董事會結構與會議": [
                    "董事會席次(席)",
                    "獨立董事席次(席)",
                    "女性董事席次(席)",
                    "公司年度召開法說會次數(次)"
                ],
                "董事參與與能力": [
                    "女性董事比率",
                    "董事出席董事會出席率",
                    "董事進修時數符合進修要點比率"
                ]
            },
            "data": {
                "董事會席次(席)": [],
                "獨立董事席次(席)": [],
                "女性董事席次(席)": [],
                "公司年度召開法說會次數(次)": [],
                "女性董事比率": [],
                "董事出席董事會出席率": [],
                "董事進修時數符合進修要點比率": []
                }
            }
        }
    }

    for i in range(len(values)): # 0 ~ 74
        value = values[i]
        j = i % 25 # 0 ~ 24
        name = names[j]
        # print(name)
        if j <= 10:
            dict_format["company"]["environmental"]["data"][name].append(value)
        elif j <= 17:
            dict_format["company"]["social"]["data"][name].append(value)
        else:
            dict_format["company"]["governance"]["data"][name].append(value)

    return json.dumps(dict_format, ensure_ascii = False, indent = 2)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Dec/2024 13:07:30] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2024 13:07:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Dec/2024 13:07:42] "GET /get_three_years_ESG_data_format HTTP/1.1" 200 -


In [13]:
import http.client

conn = http.client.HTTPConnection("127.0.0.1", 5000)  # 改用 HTTP
payload = ''
headers = {}
conn.request("GET", "/get_three_years_ESG_data_format?stock_id=2882", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{
  "company": {
    "name": "國泰金",
    "stock_id": "2882",
    "environmental": {
      "years": [
        "2021",
        "2022",
        "2023"
      ],
      "categories": {
        "溫室氣體排放量": [
          "直接(範疇一)溫室氣體排放量(公噸CO₂e)",
          "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)",
          "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
          "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)"
        ],
        "再生能源使用率": [
          "再生能源使用率"
        ],
        "用水量": [
          "用水量(公噸(t))",
          "用水密集度"
        ],
        "廢棄物重量": [
          "有害廢棄物(公噸(t))",
          "非有害廢棄物(公噸(t))",
          "總重量(有害+非有害)(公噸(t))",
          "廢棄物密集度"
        ]
      },
      "data": {
        "直接(範疇一)溫室氣體排放量(公噸CO₂e)": [
          4105.41,
          4802.45,
          5455.9747
        ],
        "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)": [
          55868.85,
          54944.72,
          51916.9489
        ],
        "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)": [
          11947333.52,
          10320638.58,
          10905030.9151
        ],
        "溫室氣體排放密集度(公噸CO₂e/百

## 三間 ESG 資料合併

In [ ]:
import re

# re.findall(r"本資料由[\u4e00-\u9fff]+公司提供", soup.prettify()) # 本資料由國泰金公司提供
start = soup.prettify().find("本資料由") + 4
end = soup.prettify().find("公司提供")
soup.prettify()[start: end]

In [ ]:
json_data = {'companies': [{'name': '台積電',
 'stock_id': '2330',
 'environmental': [{'year': '2023', 'index': '直接溫室氣體排放量(範疇一) (噸CO2e)', 'value': 2151937.0000},
 {'index': '能源間接(範疇二) (噸CO2e)', 'value': 8152497.0000},
 {'index': '其他間接(範疇三) (噸CO2e)', 'value': 6049256.0000},
 {'index': '溫室氣體排放密集度 (噸CO2e/百萬 營業額)', 'value': 6.4910},
 {'index': '再生能源使用率', 'value': 0.0920},
 {'index': '用水量 (公噸)', 'value': 82823740.0000},
 {'index': '用水密集度 (公噸/單位產品用水量（公升／十二吋晶圓當量－光罩數）)', 'value': 119.9000},
 {'index': '有害廢棄物 (公噸)', 'value': 339623.0000},
 {'index': '非有害廢棄物 (公噸)', 'value': 335080.0000},
 {'index': '總重量 (有害+非有害) (公噸)', 'value': 674703.0000},
 {'index': '廢棄物密集度 (公噸/單位晶圓委外廢棄物處理量（公斤／十二吋晶圓當量－光罩數）)', 'value': 0.9900}],
 'social': [{'index': '員工福利平均數 (仟元/人)(每年6/2起公開)', 'value': 2706.0000},
 {'index': '員工薪資平均數 (仟元/人)(每年6/2起公開)', 'value': 2463.0000},
 {'index': '非擔任主管職務之全時員工薪資平均數 (仟元/人)(每年7/1起公開)', 'value': 2425.0000},
 {'index': '非擔任主管職務之全時員工薪資中位數 (仟元/人)(每年7/1起公開)', 'value': 1851.0000},
 {'index': '管理職女性主管占比', 'value': 0.1300},
 {'index': '職業災害人數', 'value': 44.0000},
 {'index': '職業災害人數比率 (職災人數/總人數)', 'value': 0.0007}],
 'governance': [{'index': '董事會席次 (席)', 'value': 10.0000},
 {'index': '獨立董事席次 (席)', 'value': 6.0000},
 {'index': '女性董事席次(席)', 'value': 1.0000},
 {'index': '女性董事比率', 'value': 0.1000},
 {'index': '董事出席董事會出席率', 'value': 1.0000},
 {'index': '董監事進修時數符合進修要點比率', 'value': 0.4000},
 {'index': '公司年度召開法說會次數(次)', 'value': 4.0000}]},
 {'name': '鴻海',
 'stock_id': '2317',
 ...}]
}

In [ ]:
dict_format = {"companies": [{} for _ in range(len(stock_ids))]
}
for i in range(len(stock_ids)):
    dict_format["companies"][i]["name"] = company_names[i]
    dict_format["companies"][i]["stock_id"] = stock_ids[i]
    dict_format["companies"][i]["environmental"] = [{} for _ in range(11)]
    dict_format["companies"][i]["social"] = [{} for _ in range(7)]
    dict_format["companies"][i]["governance"] = [{} for _ in range(7)]

for i in range(len(names)):
    name = names[i]
    value = values[i]
    index = i % 25
    # print(index)
    if index < 11 and i < 25:
        dict_format["companies"][0]["environmental"][index] = {"year": year, "index": name, "value": value}
    elif index < 18 and i < 25:
        dict_format["companies"][0]["social"][index % 11] = {"year": year, "index": name, "value": value}
    elif index < 25 and i < 25:
        dict_format["companies"][0]["governance"][index % 18] = {"year": year, "index": name, "value": value}
    elif index < 11 and i < 50:
        dict_format["companies"][1]["environmental"][index] = {"year": year, "index": name, "value": value}
    elif index < 18 and i < 50:
        dict_format["companies"][1]["social"][index % 11] = {"year": year, "index": name, "value": value}
    elif index < 25 and i < 50:
        dict_format["companies"][1]["governance"][index % 18] = {"year": year, "index": name, "value": value}
    elif index < 11 and i < 75:
        dict_format["companies"][2]["environmental"][index] = {"year": year, "index": name, "value": value}
    elif index < 18 and i < 75:
        dict_format["companies"][2]["social"][index % 11] = {"year": year, "index": name, "value": value}
    else:
        dict_format["companies"][2]["governance"][index % 18] = {"year": year, "index": name, "value": value}

dict_format

In [ ]:
with open("2330_2317_2881_esg_1year.json", "w", encoding = "utf-8") as file:
    json.dump(dict_format, fp = file, ensure_ascii = False, indent = 2)

In [16]:
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from flask import Flask

def one_year_data():
    def call_response(stock_id, year, names, values, company_names): # 2023年(民國112年)各公司資料的 names 是一樣的
        if year == "2023":
            payload = {
            "companyCode": stock_id, # 要是字串
            "yearList": [year],
            "year": year
            }

            newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

            newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                            "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                            "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                            "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                            "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                            "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                            "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

            data = newest_response.json()["data"]
            company_name = data[0]["companyName"]
            treeModels = data[0]["treeModels"]
            company_names.append(company_name)

            temp_list = []
            for i in range(len(treeModels)):
                items = treeModels[i]["items"]
                for j in range(len(items)):
                    sections = items[j]["sections"]
                    # print(sections)
                    for k in range(len(sections)):
                        name = sections[k]["name"]
                        # print(name)
                        if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                            # print(name)
                            if name == "女性董事比率":
                                names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                                value = sections[k]["controls"][0]["value"]
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.insert(0, value)
                                values.extend(temp_list)
                            elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                                value = sections[k]["controls"][0]["value"]
                                # print(value)
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.append(value)
                            else:
                                value = sections[k]["controls"][0]["value"]
                                value = value.replace(",", "")
                                if value[-1] == "%":
                                    value = value[: -1]
                                    value = round(float(value) / 100, 4)
                                value = round(float(value), 4)
                                values.append(value)
                                names.append(name)
            return names, values, company_names
        else:
            oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                        "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                        "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                        "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]
            data = {
                "encodeURIComponent": 1,
                "step": 2,
                "co_id": stock_id, # 可以是數字或文字
                "YEAR": str(int(year) - 1911) # 可以是數字或文字
            }
            oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

            soup = bs(oldest_response.text, features = "html.parser")
            # print(soup.prettify())
            start = soup.prettify().find("本資料由") + 4
            end = soup.prettify().find("公司提供")
            company_name = soup.prettify()[start: end]
            company_names.append(company_name)
            tables = soup.find_all("table")
            target_table = tables[-2]
            trs = target_table.find_all("tr")
            for tr in trs:
                td = tr.find("td", attrs = {"align": "center"})
                if td is None:
                    continue
                text = td.text
                end = text.find("(")
                if end == -1:
                    text = text.strip()
                else:    
                    text = text[: end].strip()
                # print(text)
                # print("-" * 10)
                if text in oldest_fields:
                    field = tr.find("td", attrs = {"align": "center"}).text.strip()
                    # print(tr)
                    # print(field)
                    if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                        names.extend(["女性董事席次(席)", "女性董事比率"])
                        temp_list = tr.find_all("td", attrs = {"align": "right"})
                        # print(temp_list)
                        number = round(float(temp_list[0].text.strip()[: -1]), 4)
                        ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                        # print(number, ratio)
                        values.extend([number, ratio])
                        # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                    else: 
                        names.append(field)
                        value = tr.find("td", attrs = {"align": "right"})
                        # print(value)
                        if value is None:
                            values.append(0)
                        elif value.text == "":
                            values.append(0)  
                        else:
                            value = value.text.strip().replace(",", "")
                            if value[-1] == "%":
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                # print(value)
                            elif value[-1] == "人":
                                value = round(float(value[: -1]), 4)
                                # print(value)
                            else:
                                value = round(float(value), 4)
                            # print(value)
                            values.append(value)
            return names, values, company_names

    stock_ids = input("請輸入股票代碼，並以空格區分: ").split() # 3413 5347 6770
    year = input("請輸入西元年份: ")
    names = []
    values = []
    company_names = []

    for stock_id in stock_ids:
        time.sleep(1.5)
        names, values, company_names = call_response(stock_id, year, names, values, company_names)

    dict_format = {"companies": [{} for _ in range(len(stock_ids))]
    }
    for i in range(len(stock_ids)):
        dict_format["companies"][i]["name"] = company_names[i]
        dict_format["companies"][i]["year"] = int(year)
        dict_format["companies"][i]["stock_id"] = stock_ids[i]
        dict_format["companies"][i]["environmental"] = [{} for _ in range(11)]
        dict_format["companies"][i]["social"] = [{} for _ in range(7)]
        dict_format["companies"][i]["governance"] = [{} for _ in range(7)]

    for i in range(len(names)):
        name = names[i]
        value = values[i]
        index = i % 25
        # print(index)
        if index < 11 and i < 25:
            dict_format["companies"][0]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 25:
            dict_format["companies"][0]["social"][index % 11] = {"index": name, "value": value}
        elif index < 25 and i < 25:
            dict_format["companies"][0]["governance"][index % 18] = {"index": name, "value": value}
        elif index < 11 and i < 50:
            dict_format["companies"][1]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 50:
            dict_format["companies"][1]["social"][index % 11] = {"index": name, "value": value}
        elif index < 25 and i < 50:
            dict_format["companies"][1]["governance"][index % 18] = {"index": name, "value": value}
        elif index < 11 and i < 75:
            dict_format["companies"][2]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 75:
            dict_format["companies"][2]["social"][index % 11] = {"index": name, "value": value}
        else:
            dict_format["companies"][2]["governance"][index % 18] = {"index": name, "value": value}

    return dict_format

dict_format = one_year_data()
dict_format

{'companies': [{'name': '鴻海',
   'year': 2023,
   'stock_id': '2317',
   'environmental': [{'index': '直接(範疇一)溫室氣體排放量(公噸CO₂e)', 'value': 258108.18},
    {'index': '能源間接(範疇二)溫室氣體排放量(公噸CO₂e)', 'value': 2421127.0},
    {'index': '其他間接(範疇三)溫室氣體排放量(公噸CO₂e)', 'value': 27175368.0},
    {'index': '溫室氣體排放密集度(公噸CO₂e/百萬元營業額)', 'value': 0.435},
    {'index': '再生能源使用率', 'value': 0.6096},
    {'index': '用水量(公噸(t))', 'value': 85234800.47},
    {'index': '用水密集度', 'value': 13.83},
    {'index': '有害廢棄物(公噸(t))', 'value': 112074.3275},
    {'index': '非有害廢棄物(公噸(t))', 'value': 1274804.245},
    {'index': '總重量(有害+非有害)(公噸(t))', 'value': 1386878.5725},
    {'index': '廢棄物密集度', 'value': 0.23}],
   'social': [{'index': '員工福利平均數(每年6/2起公開)(仟元/人)', 'value': 2859.0},
    {'index': '員工薪資平均數(每年6/2起公開)(仟元/人)', 'value': 2395.0},
    {'index': '非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)', 'value': 2517.0},
    {'index': '非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)', 'value': 1772.0},
    {'index': '管理職女性主管占比', 'value': 0.2155},
    {'index':

In [13]:
with open("2317、2330、6214_2022_esg.json", "w", encoding = "utf-8") as file:
        json.dump(dict_format, fp = file, ensure_ascii = False, indent = 2)
# dict_format

In [41]:
import time
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from flask import Flask

def one_year_data():
    def call_response(stock_id, year, names, values, company_names): # 2023年(民國112年)各公司資料的 names 是一樣的
        if year == "2023":
            payload = {
            "companyCode": stock_id, # 要是字串
            "yearList": [year],
            "year": year
            }

            newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

            newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                            "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                            "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                            "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                            "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                            "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                            "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

            data = newest_response.json()["data"]
            company_name = data[0]["companyName"]
            treeModels = data[0]["treeModels"]
            company_names.append(company_name)

            temp_list = []
            for i in range(len(treeModels)):
                items = treeModels[i]["items"]
                for j in range(len(items)):
                    sections = items[j]["sections"]
                    # print(sections)
                    for k in range(len(sections)):
                        name = sections[k]["name"]
                        # print(name)
                        if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                            # print(name)
                            if name == "女性董事比率":
                                names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                                value = sections[k]["controls"][0]["value"]
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.insert(0, value)
                                values.extend(temp_list)
                            elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                                value = sections[k]["controls"][0]["value"]
                                # print(value)
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.append(value)
                            else:
                                value = sections[k]["controls"][0]["value"]
                                value = value.replace(",", "")
                                if value[-1] == "%":
                                    value = value[: -1]
                                    value = round(float(value) / 100, 4)
                                value = round(float(value), 4)
                                values.append(value)
                                names.append(name)
            return names, values, company_names
        else:
            oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                        "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                        "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                        "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]
            data = {
                "encodeURIComponent": 1,
                "step": 2,
                "co_id": stock_id, # 可以是數字或文字
                "YEAR": str(int(year) - 1911) # 可以是數字或文字
            }
            oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

            soup = bs(oldest_response.text, features = "html.parser")
            # print(soup.prettify())
            start = soup.prettify().find("本資料由") + 4
            end = soup.prettify().find("公司提供")
            company_name = soup.prettify()[start: end]
            company_names.append(company_name)
            tables = soup.find_all("table")
            target_table = tables[-2]
            trs = target_table.find_all("tr")
            for tr in trs:
                td = tr.find("td", attrs = {"align": "center"})
                if td is None:
                    continue
                text = td.text
                end = text.find("(")
                if end == -1:
                    text = text.strip()
                else:    
                    text = text[: end].strip()
                # print(text)
                # print("-" * 10)
                if text in oldest_fields:
                    field = tr.find("td", attrs = {"align": "center"}).text.strip()
                    # print(tr)
                    # print(field)
                    if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                        names.extend(["女性董事席次(席)", "女性董事比率"])
                        temp_list = tr.find_all("td", attrs = {"align": "right"})
                        # print(temp_list)
                        number = round(float(temp_list[0].text.strip()[: -1]), 4)
                        ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                        # print(number, ratio)
                        values.extend([number, ratio])
                        # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                    else: 
                        names.append(field)
                        value = tr.find("td", attrs = {"align": "right"})
                        # print(value)
                        if value is None:
                            values.append(0)
                        elif value.text == "":
                            values.append(0)  
                        else:
                            value = value.text.strip().replace(",", "")
                            if value[-1] == "%":
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                # print(value)
                            elif value[-1] == "人":
                                value = round(float(value[: -1]), 4)
                                # print(value)
                            else:
                                value = round(float(value), 4)
                            # print(value)
                            values.append(value)
            return names, values, company_names

    stock_ids = input("請輸入股票代碼，並以空格區分: ").split() # 3413 5347 6770
    year = input("請輸入西元年份: ")
    names = []
    values = []
    company_names = []

    for stock_id in stock_ids:
        time.sleep(1.5)
        names, values, company_names = call_response(stock_id, year, names, values, company_names)

    dict_format = {"companies": [{} for _ in range(len(stock_ids))]
    }
    for i in range(len(stock_ids)):
        dict_format["companies"][i]["name"] = company_names[i]
        dict_format["companies"][i]["year"] = int(year)
        dict_format["companies"][i]["stock_id"] = stock_ids[i]
        dict_format["companies"][i]["environmental"] = [{} for _ in range(11)]
        dict_format["companies"][i]["social"] = [{} for _ in range(7)]
        dict_format["companies"][i]["governance"] = [{} for _ in range(7)]

    for i in range(len(names)):
        name = names[i]
        value = values[i]
        index = i % 25
        # print(index)
        if index < 11 and i < 25:
            dict_format["companies"][0]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 25:
            dict_format["companies"][0]["social"][index % 11] = {"index": name, "value": value}
        elif index < 25 and i < 25:
            dict_format["companies"][0]["governance"][index % 18] = {"index": name, "value": value}
        elif index < 11 and i < 50:
            dict_format["companies"][1]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 50:
            dict_format["companies"][1]["social"][index % 11] = {"index": name, "value": value}
        elif index < 25 and i < 50:
            dict_format["companies"][1]["governance"][index % 18] = {"index": name, "value": value}
        elif index < 11 and i < 75:
            dict_format["companies"][2]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 75:
            dict_format["companies"][2]["social"][index % 11] = {"index": name, "value": value}
        else:
            dict_format["companies"][2]["governance"][index % 18] = {"index": name, "value": value}

    # with open("2317、2330、6214_2023_esg.json", "w", encoding = "utf-8") as file:
    #     json.dump(dict_format, fp = file, ensure_ascii = False, indent = 2)


    return json.dumps(dict_format, ensure_ascii = False, indent = 2)

dict_format = one_year_data()
print(dict_format)

{
  "companies": [
    {
      "name": "鴻海",
      "year": 2023,
      "stock_id": "2317",
      "environmental": [
        {
          "index": "直接(範疇一)溫室氣體排放量(公噸CO₂e)",
          "value": 258108.18
        },
        {
          "index": "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)",
          "value": 2421127.0
        },
        {
          "index": "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
          "value": 27175368.0
        },
        {
          "index": "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)",
          "value": 0.435
        },
        {
          "index": "再生能源使用率",
          "value": 0.6096
        },
        {
          "index": "用水量(公噸(t))",
          "value": 85234800.47
        },
        {
          "index": "用水密集度",
          "value": 13.83
        },
        {
          "index": "有害廢棄物(公噸(t))",
          "value": 112074.3275
        },
        {
          "index": "非有害廢棄物(公噸(t))",
          "value": 1274804.245
        },
        {
          "index": "總重量(有害+非有害)(公噸(t))",
          "value": 1386878.5725
       

In [ ]:
app = Flask(__name__)

@app.route("/get_one_year_data", methods = ["get"])
def one_year_data():
    def call_response(stock_id, year, names, values, company_names): # 2023年(民國112年)各公司資料的 names 是一樣的
        if year == "2023":
            payload = {
            "companyCode": stock_id, # 要是字串
            "yearList": [year],
            "year": year
            }

            newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

            newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                            "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                            "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                            "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                            "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                            "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                            "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

            data = newest_response.json()["data"]
            company_name = data[0]["companyName"]
            treeModels = data[0]["treeModels"]
            company_names.append(company_name)

            temp_list = []
            for i in range(len(treeModels)):
                items = treeModels[i]["items"]
                for j in range(len(items)):
                    sections = items[j]["sections"]
                    # print(sections)
                    for k in range(len(sections)):
                        name = sections[k]["name"]
                        # print(name)
                        if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                            # print(name)
                            if name == "女性董事比率":
                                names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                                value = sections[k]["controls"][0]["value"]
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.insert(0, value)
                                values.extend(temp_list)
                            elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                                value = sections[k]["controls"][0]["value"]
                                # print(value)
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.append(value)
                            else:
                                value = sections[k]["controls"][0]["value"]
                                value = value.replace(",", "")
                                if value[-1] == "%":
                                    value = value[: -1]
                                    value = round(float(value) / 100, 4)
                                value = round(float(value), 4)
                                values.append(value)
                                names.append(name)
            return names, values, company_names
        else:
            oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                        "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                        "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                        "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]
            data = {
                "encodeURIComponent": 1,
                "step": 2,
                "co_id": stock_id, # 可以是數字或文字
                "YEAR": str(int(year) - 1911) # 可以是數字或文字
            }
            oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

            soup = bs(oldest_response.text, features = "html.parser")
            # print(soup.prettify())
            start = soup.prettify().find("本資料由") + 4
            end = soup.prettify().find("公司提供")
            company_name = soup.prettify()[start: end]
            company_names.append(company_name)
            tables = soup.find_all("table")
            target_table = tables[-2]
            trs = target_table.find_all("tr")
            for tr in trs:
                td = tr.find("td", attrs = {"align": "center"})
                if td is None:
                    continue
                text = td.text
                end = text.find("(")
                if end == -1:
                    text = text.strip()
                else:    
                    text = text[: end].strip()
                # print(text)
                # print("-" * 10)
                if text in oldest_fields:
                    field = tr.find("td", attrs = {"align": "center"}).text.strip()
                    # print(tr)
                    # print(field)
                    if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                        names.extend(["女性董事席次(席)", "女性董事比率"])
                        temp_list = tr.find_all("td", attrs = {"align": "right"})
                        # print(temp_list)
                        number = round(float(temp_list[0].text.strip()[: -1]), 4)
                        ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                        # print(number, ratio)
                        values.extend([number, ratio])
                        # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                    else: 
                        names.append(field)
                        value = tr.find("td", attrs = {"align": "right"})
                        # print(value)
                        if value is None:
                            values.append(0)
                        elif value.text == "":
                            values.append(0)  
                        else:
                            value = value.text.strip().replace(",", "")
                            if value[-1] == "%":
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                # print(value)
                            elif value[-1] == "人":
                                value = round(float(value[: -1]), 4)
                                # print(value)
                            else:
                                value = round(float(value), 4)
                            # print(value)
                            values.append(value)
            return names, values, company_names

    stock_ids = input("請輸入股票代碼，並以空格區分: ").split()
    year = input("請輸入西元年份: ")
    names = []
    values = []
    company_names = []

    for stock_id in stock_ids:
        time.sleep(1.5)
        names, values, company_names = call_response(stock_id, year, names, values, company_names)

    dict_format = {"companies": [{} for _ in range(len(stock_ids))]}
    for i in range(len(stock_ids)):
        dict_format["companies"][i]["name"] = company_names[i]
        dict_format["companies"][i]["year"] = int(year)
        dict_format["companies"][i]["stock_id"] = stock_ids[i]
        dict_format["companies"][i]["environmental"] = [{} for _ in range(11)]
        dict_format["companies"][i]["social"] = [{} for _ in range(7)]
        dict_format["companies"][i]["governance"] = [{} for _ in range(7)]

    for i in range(len(names)):
        name = names[i]
        value = values[i]
        index = i % 25
        # print(index)
        if index < 11 and i < 25:
            dict_format["companies"][0]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 25:
            dict_format["companies"][0]["social"][index % 11] = {"index": name, "value": value}
        elif index < 25 and i < 25:
            dict_format["companies"][0]["governance"][index % 18] = {"index": name, "value": value}
        elif index < 11 and i < 50:
            dict_format["companies"][1]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 50:
            dict_format["companies"][1]["social"][index % 11] = {"index": name, "value": value}
        elif index < 25 and i < 50:
            dict_format["companies"][1]["governance"][index % 18] = {"index": name, "value": value}
        elif index < 11 and i < 75:
            dict_format["companies"][2]["environmental"][index] = {"index": name, "value": value}
        elif index < 18 and i < 75:
            dict_format["companies"][2]["social"][index % 11] = {"index": name, "value": value}
        else:
            dict_format["companies"][2]["governance"][index % 18] = {"index": name, "value": value}

    return json.dumps(dict_format, ensure_ascii = False, indent = 2)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Dec/2024 14:01:02] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [12/Dec/2024 14:01:36] "GET /get_one_year_data HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2024 14:03:25] "GET /get_one_year_data HTTP/1.1" 200 -


In [21]:
import http.client

conn = http.client.HTTPConnection("127.0.0.1", 5000)  # 改用 HTTP
payload = ''
headers = {}
conn.request("GET", "/get_one_year_ESG_data_format?stock_ids=2317,2330,6214&year=2023", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{
  "companies": [
    {
      "name": "鴻海",
      "year": 2023,
      "stock_id": "2317",
      "environmental": [
        {
          "index": "直接(範疇一)溫室氣體排放量(公噸CO₂e)",
          "value": 258108.18
        },
        {
          "index": "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)",
          "value": 2421127.0
        },
        {
          "index": "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
          "value": 27175368.0
        },
        {
          "index": "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)",
          "value": 0.435
        },
        {
          "index": "再生能源使用率",
          "value": 0.6096
        },
        {
          "index": "用水量(公噸(t))",
          "value": 85234800.47
        },
        {
          "index": "用水密集度",
          "value": 13.83
        },
        {
          "index": "有害廢棄物(公噸(t))",
          "value": 112074.3275
        },
        {
          "index": "非有害廢棄物(公噸(t))",
          "value": 1274804.245
        },
        {
          "index": "總重量(有害+非有害)(公噸(t))",
          "value": 1386878.5725
       

# ESG 資料分析

## 三年 ESG 資料分析


In [ ]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def three_years_data():
    stock_id = input("請輸入股票代碼: ")
    year = "2023"
    payload = {
    "companyCode": stock_id, # 要是字串
    "yearList": [year],
    "year": year
    }

    newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

    newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                    "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                    "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                    "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                    "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                    "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                    "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

    oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                    "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                    "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                    "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]

    data = newest_response.json()["data"]
    company_name = data[0]["companyName"]
    treeModels = data[0]["treeModels"]

    names = []
    values = []
    temp_list = []
    for i in range(len(treeModels)):
        items = treeModels[i]["items"]
        for j in range(len(items)):
            sections = items[j]["sections"]
            # print(sections)
            for k in range(len(sections)):
                name = sections[k]["name"]
                # print(name)
                if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                    # print(name)
                    if name == "女性董事比率":
                        names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                        value = sections[k]["controls"][0]["value"]
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.insert(0, value)
                        values.extend(temp_list)
                    elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                        value = sections[k]["controls"][0]["value"]
                        # print(value)
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.append(value)
                    else:
                        value = sections[k]["controls"][0]["value"]
                        value = value.replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                        value = round(float(value), 4)
                        values.append(value)
                        names.append(name)


    def call_oldest_response(stock_id, year):
        data = {
            "encodeURIComponent": 1,
            "step": 2,
            "co_id": stock_id, # 可以是數字或文字
            "YEAR": year # 可以是數字或文字
        }
        oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

        fields = []
        # values = []
        soup = bs(oldest_response.text, features = "html.parser")
        # print(soup.prettify())
        tables = soup.find_all("table")
        target_table = tables[-2]
        trs = target_table.find_all("tr")
        for tr in trs:
            td = tr.find("td", attrs = {"align": "center"})
            if td is None:
                continue
            text = td.text
            end = text.find("(")
            if end == -1:
                text = text.strip()
            else:    
                text = text[: end].strip()
            # print(text)
            # print("-" * 10)
            if text in oldest_fields:
                field = tr.find("td", attrs = {"align": "center"}).text.strip()
                # print(tr)
                # print(field)
                if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                    fields.extend(["女性董事席次(席)", "女性董事比率"])
                    temp_list = tr.find_all("td", attrs = {"align": "right"})
                    # print(temp_list)
                    number = round(float(temp_list[0].text.strip()[: -1]), 4)
                    ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                    # print(number, ratio)
                    values.extend([number, ratio])
                    # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                else: 
                    fields.append(field)
                    value = tr.find("td", attrs = {"align": "right"})
                    # print(value)
                    if value is None:
                        values.append(0)
                    elif value.text == "":
                        values.append(0)  
                    else:
                        value = value.text.strip().replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                            # print(value)
                        elif value[-1] == "人":
                            value = round(float(value[: -1]), 4)
                            # print(value)
                        else:
                            value = round(float(value), 4)
                        # print(value)
                        values.append(value)

    call_oldest_response(stock_id, int(year) - 1911 - 1)
    time.sleep(1.5)
    call_oldest_response(stock_id, int(year) - 1911 - 2)
    # print(values)
    values[: 25], values[50: ] = values[50: ], values[: 25]
    # print(values)

    df = pd.DataFrame()
    df.index = names
    df["2021年"] = values[: 25]
    df["2022年"] = values[25: 50]
    df["2023年"] = values[50: ]

    load_dotenv()
    chat_model = ChatOpenAI(model = "gpt-3.5-turbo", api_key = os.getenv("OPENAI_API_KEY"))
    prompt = ChatPromptTemplate.from_messages([
        ("system", "請您扮演一位台灣專業的 ESG 分析師"),
        ("human", "我現在有2021年至2023年的 ESG 資料，這間公司是{company_name}，欄位資訊為{names}。"),
        ("human", "請您根據以下三年的資訊，使用繁體中文各自分析 ESG 三個面向給我，並以嚴謹的角度撰寫近期的趨勢報告。\n"
         "以下為此三年的資訊：{ESG_data}")
    ])
    str_parser = StrOutputParser()

    ESG_analysis_chain = prompt | chat_model | str_parser
    ESG_response = ESG_analysis_chain.invoke({"company_name": company_name, "names": names, "ESG_data": df[["2021年", "2022年", "2023年"]]})
    
    return ESG_response

ESG_response = three_years_data()
print(ESG_response)

**精誠公司 ESG 趨勢分析報告 (2021-2023)**

**環境面向 (Environmental)**

1. **溫室氣體排放量**:
   - 2021年至2023年，精誠公司直接和間接溫室氣體排放量持續增加，尤其在直接排放量方面增幅較大。這顯示公司在減排方面可能面臨挑戰，需要加強減排措施以符合氣候變遷相關目標。

2. **再生能源使用率**:
   - 公司在再生能源使用率方面有穩定的提升，顯示其對可再生能源的利用有所改善。然而，仍有進一步提升的空間，可透過增加再生能源比例來減少對傳統能源的依賴。

3. **水資源管理**:
   - 用水量在2022年有明顯增加，但2023年又有下降趨勢。然而，用水密集度在2023年顯著下降，可能表示公司在用水效率方面有所改善。

**社會面向 (Social)**

1. **員工福利與薪資**:
   - 員工福利與薪資在過去三年持續增加，顯示公司對員工的福利與薪資有所關注。這有助於提高員工忠誠度與工作滿意度，對公司整體經營績效有正面影響。

2. **職業安全與健康**:
   - 職業災害人數在過去三年逐年減少，但職業災害人數比率仍需持續關注，公司應加強職安衛管理，確保員工工作環境安全。

**治理面向 (Governance)**

1. **董事結構與多樣性**:
   - 公司董事會席次在過去三年有些許變動，獨立董事席次有增加趨勢，顯示公司對董事結構的多元化有所關注。然而，女性董事比率在2023年有小幅下降，可能需要加強女性在決策層的代表性。

2. **公司治理與透明度**:
   - 公司年度召開法說會次數穩定，董事出席率保持高水準，顯示公司重視治理透明度與溝通。此外，董事進修時數符合進修要點比率為100%，表明公司董事對持續學習與提升專業能力有重視。

**總結與建議**:

精誠公司在環境、社會和治理三個面向均有積極的發展與改善，但仍有一些挑戰需要面對。建議公司繼續強化溫室氣體排放減量措施，提高再生能源利用率，加強水資源管理，進一步提升員工福利與薪資水平，加強職業安全與健康管理，並持續強化公司治理結構與透明度，特別是提高女性在董事會中的代表性。這將有助於公司在ESG領域的表現更上一層樓，增強企業競爭力，並符合持續永續發展的趨勢。

希望以上分析與建議能為精誠公司的ESG管理提供一些啟示，

In [37]:
import os
import time
import requests
import pandas as pd
from flask import Flask
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

app = Flask(__name__)

@app.route("/get_three_years_ESG_analysis", methods = ["get"])
def three_years_data():
    stock_id = input("請輸入股票代碼: ")
    year = "2023"
    payload = {
    "companyCode": stock_id, # 要是字串
    "yearList": [year],
    "year": year
    }

    newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

    newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                    "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                    "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                    "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                    "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                    "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                    "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

    oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                    "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                    "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                    "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]

    data = newest_response.json()["data"]
    company_name = data[0]["companyName"]
    treeModels = data[0]["treeModels"]

    names = []
    values = []
    temp_list = []
    for i in range(len(treeModels)):
        items = treeModels[i]["items"]
        for j in range(len(items)):
            sections = items[j]["sections"]
            # print(sections)
            for k in range(len(sections)):
                name = sections[k]["name"]
                # print(name)
                if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                    # print(name)
                    if name == "女性董事比率":
                        names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                        value = sections[k]["controls"][0]["value"]
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.insert(0, value)
                        values.extend(temp_list)
                    elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                        value = sections[k]["controls"][0]["value"]
                        # print(value)
                        value = value[: -1]
                        value = round(float(value) / 100, 4)
                        temp_list.append(value)
                    else:
                        value = sections[k]["controls"][0]["value"]
                        value = value.replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                        value = round(float(value), 4)
                        values.append(value)
                        names.append(name)


    def call_oldest_response(stock_id, year):
        data = {
            "encodeURIComponent": 1,
            "step": 2,
            "co_id": stock_id, # 可以是數字或文字
            "YEAR": year # 可以是數字或文字
        }
        oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

        fields = []
        # values = []
        soup = bs(oldest_response.text, features = "html.parser")
        # print(soup.prettify())
        tables = soup.find_all("table")
        target_table = tables[-2]
        trs = target_table.find_all("tr")
        for tr in trs:
            td = tr.find("td", attrs = {"align": "center"})
            if td is None:
                continue
            text = td.text
            end = text.find("(")
            if end == -1:
                text = text.strip()
            else:    
                text = text[: end].strip()
            # print(text)
            # print("-" * 10)
            if text in oldest_fields:
                field = tr.find("td", attrs = {"align": "center"}).text.strip()
                # print(tr)
                # print(field)
                if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                    fields.extend(["女性董事席次(席)", "女性董事比率"])
                    temp_list = tr.find_all("td", attrs = {"align": "right"})
                    # print(temp_list)
                    number = round(float(temp_list[0].text.strip()[: -1]), 4)
                    ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                    # print(number, ratio)
                    values.extend([number, ratio])
                    # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                else: 
                    fields.append(field)
                    value = tr.find("td", attrs = {"align": "right"})
                    # print(value)
                    if value is None:
                        values.append("Undefined")
                    elif value.text == "":
                        values.append("Undefined")  
                    else:
                        value = value.text.strip().replace(",", "")
                        if value[-1] == "%":
                            value = value[: -1]
                            value = round(float(value) / 100, 4)
                            # print(value)
                        elif value[-1] == "人":
                            value = round(float(value[: -1]), 4)
                            # print(value)
                        else:
                            value = round(float(value), 4)
                        # print(value)
                        values.append(value)

    call_oldest_response(stock_id, int(year) - 1911 - 1)
    time.sleep(1.5)
    call_oldest_response(stock_id, int(year) - 1911 - 2)
    # print(values)
    values[: 25], values[50: ] = values[50: ], values[: 25]
    # print(values)

    df = pd.DataFrame()
    df.index = names
    df["2021年"] = values[: 25]
    df["2022年"] = values[25: 50]
    df["2023年"] = values[50: ]

    load_dotenv()
    chat_model = ChatOpenAI(model = "gpt-3.5-turbo", api_key = os.getenv("OPENAI_API_KEY"))
    prompt = ChatPromptTemplate.from_messages([
        ("system", "請您扮演一位台灣專業的 ESG 分析師"),
        ("human", "我現在有2021年至2023年的 ESG 資料，這間公司是{company_name}，欄位資訊為{names}。"),
        ("human", "請您根據以下三年的資訊，使用繁體中文各自分析 ESG 三個面向給我，並以嚴謹的角度撰寫近期的趨勢報告。\n"
         "以下為此三年的資訊：{ESG_data}")
    ])
    str_parser = StrOutputParser()

    ESG_analysis_chain = prompt | chat_model | str_parser
    ESG_response = ESG_analysis_chain.invoke({"company_name": company_name, "names": names, "ESG_data": df[["2021年", "2022年", "2023年"]]})
    
    return ESG_response

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [11]:
import http.client

conn = http.client.HTTPConnection("127.0.0.1", 5000)  # 改用 HTTP
payload = ''
headers = {}
conn.request("GET", "/get_three_years_ESG_data_analysis?stock_id=6214", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

### 精誠公司 ESG 趨勢報告分析

#### 環境（Environmental）：  
1. **溫室氣體排放總量：**  
    - 從 2021 年至 2023 年，精誠公司的直接、能源間接和其他間接溫室氣體排放量有逐年增加的趨勢。
    - 在溫室氣體排放密集度方面，公司在 2022 年和 2023 年的數據顯示排放密度有所下降，顯示公司在營業額下溫室氣體排放的效率提升。
    - 值得注意的是再生能源使用率逐年上升，這顯示公司正朝著更環保和可持續的能源使用方向發展。

2. **水資源管理：**  
    - 用水量在 2022 年和 2023 年有增加，但用水密集度在 2023 年有明顯下降，顯示公司在用水效率上有改進。
    - 有害和非有害廢棄物量在 2022 年和 2023 年均有所降低，顯示公司在廢棄物管理上取得了進展。

#### 社會（Social）：  
1. **員工福利與薪資：**  
    - 員工福利平均數和員工薪資平均數在 2022 年和 2023 年均有上升，顯示公司對員工的關注和福利有提升。
    - 非擔任主管職務之全時員工薪資平均數和中位數也呈現逐年增加的趨勢，顯示公司對基層員工的薪資給付有改善。

2. **職業安全健康：**  
    - 職業災害人數在 2022 年和 2023 年有明顯下降，顯示公司對職業安全健康的關注及管理有成效。

#### 治理（Governance）：  
1. **董事會組成及運作：**  
    - 董事會席次在 2022 年有下降，但在 2023 年回升至與 2021 年相當水平。
    - 獨立董事席次有逐年增加的趨勢，顯示公司對董事會獨立性的重視。
    - 女性董事比率在 2022 年有明顯上升，但在 2023 年略微下降，公司應持續關注性別多元化議題。
    
總體而言，精誠公司在環境、社會和治理三個面向上的表現呈現出正向的趨勢，尤其在環境方面有顯著改善。公司在未來應持續關注溫室氣體排放、水資源管理、員工福利與安全等議題，並不斷提升ESG績效，以促進企業的可持續發展和長期價值創造。


## 三間 ESG 資料分析

In [16]:
import os
import time
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from flask import Flask
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def one_year_data():
    def call_response(stock_id, year, names, values, company_names): # 2023年(民國112年)各公司資料的 names 是一樣的
        if year == "2023":
            payload = {
            "companyCode": stock_id, # 要是字串
            "yearList": [year],
            "year": year
            }

            newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

            newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                            "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                            "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                            "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                            "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                            "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                            "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

            data = newest_response.json()["data"]
            company_name = data[0]["companyName"]
            treeModels = data[0]["treeModels"]
            company_names.append(company_name)

            temp_list = []
            for i in range(len(treeModels)):
                items = treeModels[i]["items"]
                for j in range(len(items)):
                    sections = items[j]["sections"]
                    # print(sections)
                    for k in range(len(sections)):
                        name = sections[k]["name"]
                        # print(name)
                        if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                            # print(name)
                            if name == "女性董事比率":
                                names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                                value = sections[k]["controls"][0]["value"]
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.insert(0, value)
                                values.extend(temp_list)
                            elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                                value = sections[k]["controls"][0]["value"]
                                # print(value)
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.append(value)
                            else:
                                value = sections[k]["controls"][0]["value"]
                                value = value.replace(",", "")
                                if value[-1] == "%":
                                    value = value[: -1]
                                    value = round(float(value) / 100, 4)
                                value = round(float(value), 4)
                                values.append(value)
                                names.append(name)
            return names, values, company_names
        else:
            oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                        "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                        "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                        "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]
            data = {
                "encodeURIComponent": 1,
                "step": 2,
                "co_id": stock_id, # 可以是數字或文字
                "YEAR": str(int(year) - 1911) # 可以是數字或文字
            }
            oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

            soup = bs(oldest_response.text, features = "html.parser")
            # print(soup.prettify())
            start = soup.prettify().find("本資料由") + 4
            end = soup.prettify().find("公司提供")
            company_name = soup.prettify()[start: end]
            company_names.append(company_name)
            tables = soup.find_all("table")
            target_table = tables[-2]
            trs = target_table.find_all("tr")
            for tr in trs:
                td = tr.find("td", attrs = {"align": "center"})
                if td is None:
                    continue
                text = td.text
                end = text.find("(")
                if end == -1:
                    text = text.strip()
                else:    
                    text = text[: end].strip()
                # print(text)
                # print("-" * 10)
                if text in oldest_fields:
                    field = tr.find("td", attrs = {"align": "center"}).text.strip()
                    # print(tr)
                    # print(field)
                    if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                        names.extend(["女性董事席次(席)", "女性董事比率"])
                        temp_list = tr.find_all("td", attrs = {"align": "right"})
                        # print(temp_list)
                        number = round(float(temp_list[0].text.strip()[: -1]), 4)
                        ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                        # print(number, ratio)
                        values.extend([number, ratio])
                        # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                    else: 
                        names.append(field)
                        value = tr.find("td", attrs = {"align": "right"})
                        # print(value)
                        if value is None:
                            values.append(0)
                        elif value.text == "":
                            values.append(0)  
                        else:
                            value = value.text.strip().replace(",", "")
                            if value[-1] == "%":
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                # print(value)
                            elif value[-1] == "人":
                                value = round(float(value[: -1]), 4)
                                # print(value)
                            else:
                                value = round(float(value), 4)
                            # print(value)
                            values.append(value)
            return names, values, company_names

    stock_ids = input("請輸入股票代碼，並以空格區分: ").split() # 3413 5347 6770
    year = input("請輸入西元年份: ")
    names = []
    values = []
    company_names = []

    for stock_id in stock_ids:
        time.sleep(1.5)
        names, values, company_names = call_response(stock_id, year, names, values, company_names)

    # return names, values, company_names

    df = pd.DataFrame()
    for i in range(len(company_names)):
        df[company_names[i]] = values[i * 25: (i + 1) * 25]
    df.index = names[: 25]

    load_dotenv()
    chat_model = ChatOpenAI(model = "gpt-3.5-turbo", api_key = os.getenv("OPENAI_API_KEY"))
    prompt = ChatPromptTemplate.from_messages([
        ("system", "請您扮演一位台灣專業的 ESG 分析師"),
        ("human", "我現在有{year}年的 ESG 資訊，有{company_amount}間公司，包含{company_names}，欄位資訊為{column_names}。"),
        ("human", "請您根據以下公司的 ESG 資訊，使用繁體中文各自分析 ESG 三個面向給我，並以嚴謹的角度撰寫近期的趨勢報告。\n"
         "以下為公司的資訊：{ESG_data}")
    ])
    str_parser = StrOutputParser()

    ESG_analysis_chain = prompt | chat_model | str_parser
    ESG_response = ESG_analysis_chain.invoke({"year": year, "company_amount": len(company_names), "company_names": company_names,
                                              "column_names": df.index, "ESG_data": df[company_names]})
    return ESG_response

ESG_response = one_year_data()
print(ESG_response)

'根據提供的公司資訊，我將分析鴻海、台積電和精誠這三家公司的ESG面向，並撰寫近期的趨勢報告。\n\n### 鴻海（Hon Hai Precision Industry Co., Ltd.）\n\n1. 環境（Environment）：\n   - 鴻海的溫室氣體排放量相對較低，但溫室氣體排放密度較高，有待改善。\n   - 用水密集度較低，顯示公司對於水資源的使用較有效率。\n   - 廢棄物密集度低，表現良好。\n\n2. 社會（Social）：\n   - 員工福利平均數和員工薪資平均數較高，顯示公司對員工的福利關懷。\n   - 職業災害人數較低，但職業災害人數比率略高。\n\n3. 治理（Governance）：\n   - 董事會席次和獨立董事席次均達到一定水準，但女性董事比率有待提高。\n\n### 台積電（Taiwan Semiconductor Manufacturing Company, Ltd.）\n\n1. 環境（Environment）：\n   - 台積電的溫室氣體排放量較高，但公司致力於再生能源使用，有潛力改善溫室氣體排放。\n   - 用水量和用水密集度均高，需關注水資源管理。\n\n2. 社會（Social）：\n   - 管理職女性主管占比較低，公司可加強性別平等政策。\n   - 員工薪資平均數較高，但非擔任主管職務之全時員工薪資中位數有提升空間。\n\n3. 治理（Governance）：\n   - 董事會結構完整，但女性董事比率較低，可加強多元化。\n\n### 精誠（Phison Electronics Corp.）\n\n1. 環境（Environment）：\n   - 精誠的溫室氣體排放量相對較低，但公司對再生能源使用率和用水量有進步空間。\n   - 廢棄物密集度低，表現良好。\n\n2. 社會（Social）：\n   - 公司員工薪資平均數和非擔任主管職務之全時員工薪資中位數較低，可關注員工福利。\n   - 女性董事比率較低，公司治理需更多性別多元化。\n\n3. 治理（Governance）：\n   - 董事會出席率和董事進修時數表現良好，但女性董事席次有提升空間。\n\n### 報告結論：\n鴻海在社會和治理方面表現較為穩健，但在環境方面仍有改善空間。台積電在環境和社會方面有部分挑戰，治理表現較佳。精

In [17]:
print(ESG_response)

根據提供的公司資訊，我將分析鴻海、台積電和精誠這三家公司的ESG面向，並撰寫近期的趨勢報告。

### 鴻海（Hon Hai Precision Industry Co., Ltd.）

1. 環境（Environment）：
   - 鴻海的溫室氣體排放量相對較低，但溫室氣體排放密度較高，有待改善。
   - 用水密集度較低，顯示公司對於水資源的使用較有效率。
   - 廢棄物密集度低，表現良好。

2. 社會（Social）：
   - 員工福利平均數和員工薪資平均數較高，顯示公司對員工的福利關懷。
   - 職業災害人數較低，但職業災害人數比率略高。

3. 治理（Governance）：
   - 董事會席次和獨立董事席次均達到一定水準，但女性董事比率有待提高。

### 台積電（Taiwan Semiconductor Manufacturing Company, Ltd.）

1. 環境（Environment）：
   - 台積電的溫室氣體排放量較高，但公司致力於再生能源使用，有潛力改善溫室氣體排放。
   - 用水量和用水密集度均高，需關注水資源管理。

2. 社會（Social）：
   - 管理職女性主管占比較低，公司可加強性別平等政策。
   - 員工薪資平均數較高，但非擔任主管職務之全時員工薪資中位數有提升空間。

3. 治理（Governance）：
   - 董事會結構完整，但女性董事比率較低，可加強多元化。

### 精誠（Phison Electronics Corp.）

1. 環境（Environment）：
   - 精誠的溫室氣體排放量相對較低，但公司對再生能源使用率和用水量有進步空間。
   - 廢棄物密集度低，表現良好。

2. 社會（Social）：
   - 公司員工薪資平均數和非擔任主管職務之全時員工薪資中位數較低，可關注員工福利。
   - 女性董事比率較低，公司治理需更多性別多元化。

3. 治理（Governance）：
   - 董事會出席率和董事進修時數表現良好，但女性董事席次有提升空間。

### 報告結論：
鴻海在社會和治理方面表現較為穩健，但在環境方面仍有改善空間。台積電在環境和社會方面有部分挑戰，治理表現較佳。精誠在環境和治理方面表現不錯，但在社會方面有些許改進空間。以上三家公司在ESG方面都有機會

In [15]:
df = pd.DataFrame()
for i in range(len(company_names)):
    df[company_names[i]] = values[i * 25: (i + 1) * 25]
df.index = names[: 25]
df

,鴻海,台積電,精誠
直接溫室氣體排放量(範疇一) (噸CO2e),2.379460e+05,2.151937e+06,7.2500
能源間接(範疇二) (噸CO2e),5.982836e+06,8.152497e+06,2779.5100
其他間接(範疇三) (噸CO2e),2.402574e+07,6.049256e+06,566.8800
溫室氣體排放密集度 (噸CO2e/百萬新台幣營業額),1.040000e+00,6.491000e+00,2.9600
再生能源使用率,5.170000e-02,9.200000e-02,0.0130
用水量 (公噸),8.770600e+07,8.282374e+07,17426.0000
用水密集度 (公噸/百萬新台幣營業額),1.460000e+01,1.199000e+02,15.3700
有害廢棄物 (公噸),4.534430e+05,3.396230e+05,0.0000
非有害廢棄物 (公噸),5.483280e+05,3.350800e+05,0.0000
總重量 (有害+非有害) (公噸),1.001771e+06,6.747030e+05,66.9000


In [10]:
df[company_names]

,鴻海,台積電,精誠
直接(範疇一)溫室氣體排放量(公噸CO₂e),2.581082e+05,1.596031e+06,20.4516
能源間接(範疇二)溫室氣體排放量(公噸CO₂e),2.421127e+06,1.018739e+07,2810.9551
其他間接(範疇三)溫室氣體排放量(公噸CO₂e),2.717537e+07,7.616655e+06,643.4910
溫室氣體排放密集度(公噸CO₂e/百萬元營業額),4.350000e-01,5.223900e+00,0.0988
再生能源使用率,6.096000e-01,1.116000e-01,0.0205
用水量(公噸(t)),8.523480e+07,1.010016e+08,22375.0000
用水密集度,1.383000e+01,1.764318e+02,0.6359
有害廢棄物(公噸(t)),1.120743e+05,3.712360e+05,0.0000
非有害廢棄物(公噸(t)),1.274804e+06,2.856050e+05,43.6000
總重量(有害+非有害)(公噸(t)),1.386879e+06,6.568410e+05,43.6000


In [35]:
df.columns.name = "測試"
df

測試,鴻海,台積電,精誠
直接(範疇一)溫室氣體排放量(公噸CO₂e),2.581082e+05,1.596031e+06,20.4516
能源間接(範疇二)溫室氣體排放量(公噸CO₂e),2.421127e+06,1.018739e+07,2810.9551
其他間接(範疇三)溫室氣體排放量(公噸CO₂e),2.717537e+07,7.616655e+06,643.4910
溫室氣體排放密集度(公噸CO₂e/百萬元營業額),4.350000e-01,5.223900e+00,0.0988
再生能源使用率,6.096000e-01,1.116000e-01,0.0205
用水量(公噸(t)),8.523480e+07,1.010016e+08,22375.0000
用水密集度,1.383000e+01,1.764318e+02,0.6359
有害廢棄物(公噸(t)),1.120743e+05,3.712360e+05,0.0000
非有害廢棄物(公噸(t)),1.274804e+06,2.856050e+05,43.6000
總重量(有害+非有害)(公噸(t)),1.386879e+06,6.568410e+05,43.6000


In [27]:
names[: 25]

['直接(範疇一)溫室氣體排放量(公噸CO₂e)',
 '能源間接(範疇二)溫室氣體排放量(公噸CO₂e)',
 '其他間接(範疇三)溫室氣體排放量(公噸CO₂e)',
 '溫室氣體排放密集度(公噸CO₂e/百萬元營業額)',
 '再生能源使用率',
 '用水量(公噸(t))',
 '用水密集度',
 '有害廢棄物(公噸(t))',
 '非有害廢棄物(公噸(t))',
 '總重量(有害+非有害)(公噸(t))',
 '廢棄物密集度',
 '員工福利平均數(每年6/2起公開)(仟元/人)',
 '員工薪資平均數(每年6/2起公開)(仟元/人)',
 '非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)',
 '非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)',
 '管理職女性主管占比',
 '職業災害人數(人)',
 '職業災害人數比率',
 '董事會席次(席)',
 '獨立董事席次(席)',
 '女性董事席次(席)',
 '女性董事比率',
 '董事出席董事會出席率',
 '董事進修時數符合進修要點比率',
 '公司年度召開法說會次數(次)']

In [24]:
values

[258108.18,
 2421127.0,
 27175368.0,
 0.435,
 0.6096,
 85234800.47,
 13.83,
 112074.3275,
 1274804.245,
 1386878.5725,
 0.23,
 2859.0,
 2395.0,
 2517.0,
 1772.0,
 0.2155,
 31.0,
 0.0077,
 8.0,
 5.0,
 2.0,
 0.25,
 0.9,
 1.0,
 4.0,
 1596031.2036,
 10187386.7231,
 7616654.7896,
 5.2239,
 0.1116,
 101001617.0,
 176.4318,
 371236.0,
 285605.0,
 656841.0,
 1.17,
 3015.0,
 2737.0,
 2842.0,
 2248.0,
 0.141,
 48.0,
 0.0007,
 10.0,
 6.0,
 1.0,
 0.1,
 0.94,
 0.2,
 4.0,
 20.4516,
 2810.9551,
 643.491,
 0.0988,
 0.0205,
 22375.0,
 0.6359,
 0.0,
 43.6,
 43.6,
 0.0012,
 1213.0,
 987.0,
 1006.0,
 822.0,
 0.26,
 1.0,
 0.0002,
 13.0,
 4.0,
 2.0,
 0.1538,
 0.96,
 1.0,
 4.0]

In [25]:
company_names

['鴻海', '台積電', '精誠']

In [27]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from bs4 import BeautifulSoup as bs
from flask import Flask
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

app = Flask(__name__)

@app.route("/get_one_year_ESG_data_analysis", methods = ["get"])
def one_year_data():
    def call_response(stock_id, year, names, values, company_names): # 2023年(民國112年)各公司資料的 names 是一樣的
        if year == "2023":
            payload = {
            "companyCode": stock_id, # 要是字串
            "yearList": [year],
            "year": year
            }

            newest_response = requests.post("https://esggenplus.twse.com.tw/api/api/mopsEsg/singleCompanyData", json = payload)

            newest_fields = ["直接(範疇一)溫室氣體排放量(公噸CO₂e)", "能源間接(範疇二)溫室氣體排放量(公噸CO₂e)", "其他間接(範疇三)溫室氣體排放量(公噸CO₂e)",
                            "溫室氣體排放密集度(公噸CO₂e/百萬元營業額)", "再生能源使用率", "用水量(公噸(t))", "用水密集度", "有害廢棄物(公噸(t))",
                            "非有害廢棄物(公噸(t))", "總重量(有害+非有害)(公噸(t))", "廢棄物密集度", "員工福利平均數(每年6/2起公開)(仟元/人)",
                            "員工薪資平均數(每年6/2起公開)(仟元/人)", "非擔任主管職務之全時員工薪資平均數(每年7/1起公開)(仟元/人)", 
                            "非擔任主管職務之全時員工薪資中位數(每年7/1起公開)(仟元/人)", "管理職女性主管占比", "職業災害人數(人)", 
                            "職業災害人數比率", "董事會席次(席)", "獨立董事席次(席)", "女性董事席次(席)", "女性董事比率", "董事出席董事會出席率", 
                            "董事進修時數符合進修要點比率", "公司年度召開法說會次數(次)"] # 抓sections裡的name

            data = newest_response.json()["data"]
            company_name = data[0]["companyName"]
            treeModels = data[0]["treeModels"]
            company_names.append(company_name)

            temp_list = []
            for i in range(len(treeModels)):
                items = treeModels[i]["items"]
                for j in range(len(items)):
                    sections = items[j]["sections"]
                    # print(sections)
                    for k in range(len(sections)):
                        name = sections[k]["name"]
                        # print(name)
                        if name in newest_fields: # 女性董事席次(席) 女性董事比率 女性董事席次及比率 確定後面有沒有空格
                            # print(name)
                            if name == "女性董事比率":
                                names.extend([name, "董事出席董事會出席率", "董事進修時數符合進修要點比率"])
                                value = sections[k]["controls"][0]["value"]
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.insert(0, value)
                                values.extend(temp_list)
                            elif name == "董事出席董事會出席率" or name == "董事進修時數符合進修要點比率":
                                value = sections[k]["controls"][0]["value"]
                                # print(value)
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                temp_list.append(value)
                            else:
                                value = sections[k]["controls"][0]["value"]
                                value = value.replace(",", "")
                                if value[-1] == "%":
                                    value = value[: -1]
                                    value = round(float(value) / 100, 4)
                                value = round(float(value), 4)
                                values.append(value)
                                names.append(name)
            return names, values, company_names
        else:
            oldest_fields = ["直接溫室氣體排放量", "能源間接", "其他間接", "溫室氣體排放密集度", "再生能源使用率", "用水量", "用水密集度", "有害廢棄物", 
                        "非有害廢棄物", "總重量", "廢棄物密集度", "員工福利平均數", "員工薪資平均數", "非擔任主管職務之全時員工薪資平均數",
                        "非擔任主管職務之全時員工薪資中位數", "管理職女性主管占比", "職業災害人數", "職業災害人數比率", "董事會席次", "獨立董事席次",
                        "女性董事席次及比率", "董事出席董事會出席率", "董監事進修時數符合進修要點比率", "公司年度召開法說會次數"]
            data = {
                "encodeURIComponent": 1,
                "step": 2,
                "co_id": stock_id, # 可以是數字或文字
                "YEAR": str(int(year) - 1911) # 可以是數字或文字
            }
            oldest_response = requests.post("https://mops.twse.com.tw/mops/web/t214sb01", data = data)

            soup = bs(oldest_response.text, features = "html.parser")
            # print(soup.prettify())
            start = soup.prettify().find("本資料由") + 4
            end = soup.prettify().find("公司提供")
            company_name = soup.prettify()[start: end]
            company_names.append(company_name)
            tables = soup.find_all("table")
            target_table = tables[-2]
            trs = target_table.find_all("tr")
            for tr in trs:
                td = tr.find("td", attrs = {"align": "center"})
                if td is None:
                    continue
                text = td.text
                end = text.find("(")
                if end == -1:
                    text = text.strip()
                else:    
                    text = text[: end].strip()
                # print(text)
                # print("-" * 10)
                if text in oldest_fields:
                    field = tr.find("td", attrs = {"align": "center"}).text.strip()
                    # print(tr)
                    # print(field)
                    if field == "女性董事席次及比率": # 女性董事席次(席) 女性董事比率 女性董事席次及比率
                        names.extend(["女性董事席次(席)", "女性董事比率"])
                        temp_list = tr.find_all("td", attrs = {"align": "right"})
                        # print(temp_list)
                        number = round(float(temp_list[0].text.strip()[: -1]), 4)
                        ratio = round(float(temp_list[1].text.strip()[: -1]) / 100, 4)
                        # print(number, ratio)
                        values.extend([number, ratio])
                        # print(f"{temp_list[0].text.strip()}({temp_list[1].text.strip()})")
                    else: 
                        names.append(field)
                        value = tr.find("td", attrs = {"align": "right"})
                        # print(value)
                        if value is None:
                            values.append(0)
                        elif value.text == "":
                            values.append(0)  
                        else:
                            value = value.text.strip().replace(",", "")
                            if value[-1] == "%":
                                value = value[: -1]
                                value = round(float(value) / 100, 4)
                                # print(value)
                            elif value[-1] == "人":
                                value = round(float(value[: -1]), 4)
                                # print(value)
                            else:
                                value = round(float(value), 4)
                            # print(value)
                            values.append(value)
            return names, values, company_names

    stock_ids = input("請輸入股票代碼，並以空格區分: ").split() # 3413 5347 6770
    year = input("請輸入西元年份: ")
    names = []
    values = []
    company_names = []

    for stock_id in stock_ids:
        time.sleep(1.5)
        names, values, company_names = call_response(stock_id, year, names, values, company_names)

    # return names, values, company_names

    df = pd.DataFrame()
    for i in range(len(company_names)):
        df[company_names[i]] = values[i * 25: (i + 1) * 25]
    df.index = names[: 25]

    load_dotenv()
    chat_model = ChatOpenAI(model = "gpt-3.5-turbo", api_key = os.getenv("OPENAI_API_KEY"))
    prompt = ChatPromptTemplate.from_messages([
        ("system", "請您扮演一位台灣專業的 ESG 分析師"),
        ("human", "我現在有{year}年的 ESG 資訊，有{company_amount}間公司，包含{company_names}，欄位資訊為{column_names}。"),
        ("human", "請您根據以下公司的 ESG 資訊，使用繁體中文各自分析 ESG 三個面向給我，並以嚴謹的角度撰寫近期的趨勢報告。\n"
         "以下為公司的資訊：{ESG_data}")
    ])
    str_parser = StrOutputParser()

    ESG_analysis_chain = prompt | chat_model | str_parser
    ESG_response = ESG_analysis_chain.invoke({"year": year, "company_amount": len(company_names), "company_names": company_names,
                                              "column_names": df.index, "ESG_data": df[company_names]})
    return ESG_response

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Dec/2024 14:42:10] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2024 14:42:38] "GET /get_one_year_ESG_data_analysis HTTP/1.1" 200 -


In [ ]:
import http.client

conn = http.client.HTTPConnection("127.0.0.1", 5000)  # 改用 HTTP
payload = ''
headers = {}
conn.request("GET", "/get_one_year_ESG_data_analysis?stock_ids=2330,2344,6770&year=2023", payload, headers) # 2330,2344,6770
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

根據提供的資訊，我將針對台積電、華邦電和力積電這三家公司的ESG資訊進行比較和分析。請注意，ESG（環境、社會、公司治理）是企業永續經營的重要面向，因此在分析時除了單純的數據比較外，還需考慮公司在永續發展方面的績效和政策。

### 環境（Environmental）面向比較分析：
1. **溫室氣體排放量**：從資料可看出，台積電的溫室氣體排放量遠高於華邦電和力積電，可能與其業務規模和生產量有關。華邦電和力積電在這方面表現相對較好。
2. **再生能源使用率**：台積電的再生能源使用率明顯高於華邦電和力積電，顯示其在能源使用方面有較好的表現。
3. **用水量和廢棄物處理**：華邦電和力積電的用水密集度和廢棄物密集度較低，顯示在水資源利用和廢棄物處理方面有較好的管理。

### 社會（Social）面向比較分析：
1. **員工福利和薪資**：台積電的員工福利平均數和薪資平均數明顯高於華邦電和力積電，顯示其對員工福利的投入較大。
2. **職業安全**：華邦電和力積電的職業災害人數和比率較低，可能代表其在職業安全管理上較為嚴謹。

### 公司治理（Governance）面向比較分析：
1. **董事結構**：台積電的董事會席次和獨立董事席次較多，顯示其公司治理結構較為完善。
2. **女性董事比率**：華邦電的女性董事比率最高，顯示其在性別多元化方面表現較好。
3. **公司透明度**：力積電在公司年度召開法說會次數上領先，顯示其對於與投資人溝通的重視。

綜合以上分析，台積電在環境和社會方面表現較為出色，但在公司治理方面仍有一些可以改進的空間；華邦電在社會和公司治理方面表現較好，但在環境方面有一些待提升的項目；力積電則在公司治理和公司透明度方面表現出眾。這些資訊可以幫助投資者更全面地評估這三家公司在ESG方面的表現，並作出更明智的投資決策。
